# delete everything from the current folder

In [1]:
import os

os.system("rm -rf *")

0

# import raw dataset

In [2]:
from google.colab import files
print("Upload emr_alpha.csv :")
raw_claims_data_csv = files.upload()
print("\nUpload emr_beta.json :")
raw_claims_data_json = files.upload()

Upload emr_alpha.csv :


Saving emr_alpha.csv to emr_alpha.csv

Upload emr_beta.json :


Saving emr_beta.json to emr_beta.json


# import libraries

In [3]:
import pandas as pd
import json
from datetime import datetime

# helper functions

In [4]:
def normalize_date(date_str):
    """Convert date to ISO format (YYYY-MM-DDTHH:MM:SS)"""
    if not date_str or str(date_str).lower() == "none":
        return None
    return pd.to_datetime(date_str).strftime("%Y-%m-%dT%H:%M:%S")

In [5]:
def clean_value(value):
    """Standardize null handling"""
    if pd.isnull(value) or str(value).strip().lower() in ["", "none", "null"]:
        return None
    return str(value).strip()

In [6]:
def clean_reason(value):
    """Normalize denial_reason with title casing"""
    cleaned = clean_value(value)
    return cleaned.title() if cleaned else None

# load + normalize csv

In [7]:
df_alpha = pd.read_csv("emr_alpha.csv")

alpha_records = []
for _, row in df_alpha.iterrows():
    alpha_records.append({
        "claim_id": clean_value(row.get("claim_id")),
        "patient_id": clean_value(row.get("patient_id")),
        "procedure_code": clean_value(row.get("procedure_code")),
        "denial_reason": clean_reason(row.get("denial_reason")),
        "status": clean_value(row.get("status")).lower() if clean_value(row.get("status")) else None,
        "submitted_at": normalize_date(row.get("submitted_at")),
        "source_system": "alpha"
    })

# load + normalize json

In [8]:
with open("emr_beta.json", "r") as f:
    beta_data = json.load(f)

beta_records = []
for row in beta_data:
    beta_records.append({
        "claim_id": clean_value(row.get("id")),
        "patient_id": clean_value(row.get("member")),
        "procedure_code": clean_value(row.get("code")),
        "denial_reason": clean_reason(row.get("error_msg")),
        "status": clean_value(row.get("status")).lower() if clean_value(row.get("status")) else None,
        "submitted_at": normalize_date(row.get("date")),
        "source_system": "beta"
    })

# combine into unified dataset

In [9]:
unified_records = alpha_records + beta_records

In [10]:
with open("unified_claims.json", "w") as out:
    json.dump(unified_records, out, indent=2)

print("☑️Normalization complete. Records saved to unified_claims.json")

☑️Normalization complete. Records saved to unified_claims.json


# inspect new dataset

In [11]:
pd.read_json("unified_claims.json")

,claim_id,patient_id,procedure_code,denial_reason,status,submitted_at,source_system
0,A123,P001,99213,Missing Modifier,denied,2025-07-01,alpha
1,A124,P002,99214,Incorrect Npi,denied,2025-07-10,alpha
2,A125,None,99215,Authorization Expired,denied,2025-07-05,alpha
3,A126,P003,99381,None,approved,2025-07-15,alpha
4,A127,P004,99401,Prior Auth Required,denied,2025-07-20,alpha
5,B987,P010,99213,Incorrect Provider Type,denied,2025-07-03,beta
6,B988,P011,99214,Missing Modifier,denied,2025-07-09,beta
7,B989,P012,99215,None,approved,2025-07-10,beta
8,B990,None,99401,Incorrect Procedure,denied,2025-07-01,beta


# resubmission eligibility

### assuming today's date as 30/07/2025

In [12]:
TODAY = datetime(2025, 7, 30)

In [13]:
RETRYABLE_REASONS = {"missing modifier", "incorrect npi", "prior auth required"}
NON_RETRYABLE_REASONS = {"authorization expired", "incorrect provider type"}
AMBIGUOUS_REASONS = {"incorrect procedure", "form incomplete", "not billable", None}

In [14]:
def classify_denial_reason(reason):
    """Classify denial reason into retryable/non-retryable/ambiguous"""
    if reason is None:
        return "non_retryable_inferred"   # treating null as non-retryable
    normalized_reason = reason.lower().strip()
    if normalized_reason in RETRYABLE_REASONS:
        return "retryable"
    elif normalized_reason in NON_RETRYABLE_REASONS:
        return "non_retryable"
    elif normalized_reason in [r for r in [r for r in AMBIGUOUS_REASONS if r is not None]]:
        if normalized_reason == "incorrect procedure":
            return "retryable_inferred"  # hardcoded as retryable
        else:
            return "non_retryable_inferred"
    return "unknown"

In [15]:
resubmission_candidates = []
excluded_claims_log = []
total_claims_processed = 0
claims_by_source = {"alpha": 0, "beta": 0}

for record in unified_records:
    total_claims_processed += 1
    claims_by_source[record["source_system"]] += 1

    claim_id = record["claim_id"]
    patient_id = record["patient_id"]
    status = record["status"]
    denial_reason = record["denial_reason"]
    submitted_at_str = record["submitted_at"]
    source_system = record["source_system"]

    # status must be denied
    if status != "denied":
        excluded_claims_log.append({"claim_id": claim_id, "reason": "Not denied", "source_system": source_system})
        continue

    # id must not be null
    if patient_id is None:
        excluded_claims_log.append({"claim_id": claim_id, "reason": "Patient ID is null", "source_system": source_system})
        continue

    # submitted more than 7 days ago
    try:
        submitted_dt = datetime.fromisoformat(submitted_at_str) if submitted_at_str else None
    except ValueError:
        submitted_dt = None

    if submitted_dt is None or (TODAY - submitted_dt).days <= 7:
        excluded_claims_log.append({"claim_id": claim_id, "reason": "Submitted ≤ 7 days ago or invalid date", "source_system": source_system})
        continue

    # denial reason must be retryable
    reason_classification = classify_denial_reason(denial_reason)
    if not reason_classification.startswith("retryable"):
        excluded_claims_log.append({"claim_id": claim_id, "reason": f"Denial reason '{denial_reason}' not retryable", "source_system": source_system})
        continue

    # eligible part
    resubmission_candidates.append({
        "claim_id": claim_id,
        "resubmission_reason": denial_reason if denial_reason else "Ambiguous/Null Reason",
        "source_system": source_system,
        "recommended_changes": "Review claim details and resubmit."
    })

# output

In [16]:
print("\n--- Resubmission Candidates ---")
print(json.dumps(resubmission_candidates, indent=2))

print("\n--- Processing Metrics ---")
print(f"Total claims processed: {total_claims_processed}")
print(f"Claims from EMR Alpha: {claims_by_source['alpha']}")
print(f"Claims from EMR Beta: {claims_by_source['beta']}")
print(f"Claims flagged for resubmission: {len(resubmission_candidates)}")
print(f"Claims excluded: {len(excluded_claims_log)}")

print("\n--- Excluded Claims Log ---")
print(json.dumps(excluded_claims_log, indent=2))

with open("resubmission_candidates.json", "w") as f:
    json.dump(resubmission_candidates, f, indent=2)

print("\n☑️Output saved to resubmission_candidates.json")


--- Resubmission Candidates ---
[
  {
    "claim_id": "A123",
    "resubmission_reason": "Missing Modifier",
    "source_system": "alpha",
    "recommended_changes": "Review claim details and resubmit."
  },
  {
    "claim_id": "A124",
    "resubmission_reason": "Incorrect Npi",
    "source_system": "alpha",
    "recommended_changes": "Review claim details and resubmit."
  },
  {
    "claim_id": "A127",
    "resubmission_reason": "Prior Auth Required",
    "source_system": "alpha",
    "recommended_changes": "Review claim details and resubmit."
  },
  {
    "claim_id": "B988",
    "resubmission_reason": "Missing Modifier",
    "source_system": "beta",
    "recommended_changes": "Review claim details and resubmit."
  }
]

--- Processing Metrics ---
Total claims processed: 9
Claims from EMR Alpha: 5
Claims from EMR Beta: 4
Claims flagged for resubmission: 4
Claims excluded: 5

--- Excluded Claims Log ---
[
  {
    "claim_id": "A125",
    "reason": "Patient ID is null",
    "source_syste

# inspecting resubmission_candidates.json

In [17]:
pd.read_json("resubmission_candidates.json")

,claim_id,resubmission_reason,source_system,recommended_changes
0,A123,Missing Modifier,alpha,Review claim details and resubmit.
1,A124,Incorrect Npi,alpha,Review claim details and resubmit.
2,A127,Prior Auth Required,alpha,Review claim details and resubmit.
3,B988,Missing Modifier,beta,Review claim details and resubmit.
